<a href="https://colab.research.google.com/github/SherinBojappa/prompting/blob/main/opt_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# use huggingface bits and bytes to get 8 bit model
# https://huggingface.co/docs/transformers/perf_infer_gpu_one
# paper - https://arxiv.org/abs/2208.07339

In [2]:
#load latest version of bits and bytes from here
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.8 MB 163 kB/s 


In [21]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 66.8 MB/s 
     |████████████████████████████████| 175 kB 4.6 MB/s 


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 17 16:36:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
name = "facebook/opt-13b"
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(name)

Downloading:   0%|          | 0.00/719 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.87G [00:00<?, ?B/s]

...

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA path found: /usr/local/cuda/lib64/libcudart.so
CUDA_SETUP: Detected CUDA version 112
CUDA_SETUP: Loading binary /usr/local/lib/python3.7/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


Downloading:   0%|          | 0.00/721 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [6]:
def generate_from_model(model, tokenizer, max_new_tokens, text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda(), max_new_tokens=max_new_tokens)
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [ ]:
# get response from opt, this is the initial bootstrapping code different for different datasets
text = 'How would you tell if hypothesis is enatiled by premise'

In [ ]:
template_entailment = generate_from_model(model_8bit, tokenizer, 100, text)

In [ ]:
template_entailment

'How would you tell if hypothesis is enatiled by premise?\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI have a hypothesis that I want to test. I have a premise that I want to test. I have a conclusion that I want to test.\n\nI'

In [7]:
text = 'What is entailment in natural language?'
template_entailment = generate_from_model(model_8bit, tokenizer, 50, text)


In [8]:
template_entailment

'What is entailment in natural language?\n\nThe term entailment is used in natural language to refer to the relationship between two sentences. The relationship is a logical one, and it is a relationship between two sentences that are logically related.\n\nThe term entailment is used in natural'

In [10]:
text = 'What are good labels for the task of entailment?'
template_label = generate_from_model(model_8bit, tokenizer, 50, text)

In [11]:
template_label

'What are good labels for the task of entailment?\n\nI\'m trying to find a good label for the task of entailment. I\'m thinking of something like "the task of determining whether a sentence is true or false".\n\nI\'m not sure if this is the right place to ask'

In [12]:
text = 'What are good labels to identify relationship between two sentences?'
template_label = generate_from_model(model_8bit, tokenizer, 50, text)

In [13]:
template_label

'What are good labels to identify relationship between two sentences?\n\nI have a problem with identifying the relationship between two sentences. I have a sentence which is a question and I want to identify the relationship between the two sentences.\n\nI have a problem with identifying the relationship between two sentences. I have'

In [24]:
text = 'What is sentiment analysis in natural language?'
template_sentiment = generate_from_model(model_8bit, tokenizer, 50, text)


In [25]:
template_sentiment

'What is sentiment analysis in natural language?\n\nSentiment analysis is the process of identifying the emotional state of a text. It is a sub-field of natural language processing (NLP) that deals with the analysis of text.\n\nSentiment analysis is a sub-field of'

In [ ]:
text = 'What is inference in natural language?'
template_inference = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_inference

'What is inference in natural language?\n\nInference is the process of drawing conclusions from the evidence available. Inference is a process of reasoning that is used in many different areas of science and technology. Inference is also used in many areas of social science and humanities.\n\n'

In [ ]:
text = 'What is coreference resolution in natural language?'
template_coref = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_coref

'What is coreference resolution in natural language?\n\nCoreference resolution is the process of determining the correct reference in a sentence.\n\nCoreference resolution is the process of determining the correct reference in a sentence.\n\nCoreference resolution is the process of determining the correct reference in a sentence'

In [ ]:
text = 'What is word sense disambiguation in natural language?'
template_wsd = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_wsd

'What is word sense disambiguation in natural language?\n\nWord sense disambiguation is the process of determining the meaning of a word in a sentence. It is a process that is used in natural language processing.\n\nWord sense disambiguation is a process that is used in natural'

In [ ]:
text = 'What is sentence completion in natural language?'
template_sent_comp = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_sent_comp

'What is sentence completion in natural language?\n\nSentence completion is the process of completing a sentence by adding a word or phrase to it. Sentence completion is a natural language processing task.\n\nSentence completion is a natural language processing task.\n\nSentence completion is the'

In [ ]:
text = 'What is paraphrase identification in natural language?'
template_paraphrase_id = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_paraphrase_id

'What is paraphrase identification in natural language?\n\nParaphrase identification is the process of identifying a paraphrase of a text. Paraphrases are a type of paraphrase that is not a direct quotation of the original text. Paraphrases are often used to make a point'

In [ ]:
text = 'What is summarization in natural language?'
template_summarize = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_summarize

'What is summarization in natural language?\n\nSummarization is the process of extracting the main ideas from a text and presenting them in a more compact form. Summarization is a very important part of the process of information retrieval.\n\nThe main idea of summarization is to'

In [ ]:
text = 'What is question answering in natural language?'
template_qa = generate_from_model(model_8bit, tokenizer, 50, text)

In [ ]:
template_qa

'What is question answering in natural language?\n\nQuestion answering in natural language is the process of answering questions posed in natural language. The process of question answering in natural language is a subset of the process of question answering in machine language.\n\nQuestion answering in natural language is the process of'

In [ ]:
# rte, prompt formatted similar to gpt-3 style for anli with added instruction from the LM at the top, demonstrations with premise and hypothesis and then label

In [14]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 53.0 MB/s 
     |████████████████████████████████| 212 kB 50.3 MB/s 
     |████████████████████████████████| 127 kB 52.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [15]:
import datasets as datasets

In [16]:
dataset = 'rte'

In [17]:
# expand to other datasets here, we will be reporting performance on the dev sets going forward.
if dataset == 'rte':
  train_set = datasets.load_dataset('super_glue',dataset, split='train')
  dev_set = datasets.load_dataset('super_glue', dataset, split='validation')

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/rte/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed. Subsequent calls will reuse this data.


In [22]:
import pandas as pd
data_df = pd.DataFrame(dev_set)

In [23]:
data_df.head()

,premise,hypothesis,idx,label
0,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,0,1
1,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,1,0
2,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,2,1
3,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,3,1
4,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,4,0


In [32]:
def get_len(premise, hypothesis):
  len_premise = len(premise.split(" "))
  len_hypothesis = len(hypothesis.split(" "))
  len_example = len_premise+len_hypothesis

  return len_example

In [33]:
data_df['ex_len'] = data_df.apply(lambda x: get_len(x['premise'], x['hypothesis']), axis=1)

In [34]:
data_df.head()

,premise,hypothesis,idx,label,ex_len
0,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,0,1,28
1,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,1,0,38
2,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,2,1,137
3,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,3,1,108
4,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,4,0,35


In [35]:
# max length for rte is 173 so we do not need to worry about having less number of demonstrations 
data_df['ex_len'].max()

173

In [ ]:
# extend the code for the entire dataset.